<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [8]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import numpy as np
from scipy.optimize import root, minimize

In [9]:
p=Props(['Ethanol','Water'])

In [10]:
d=DotDict()

d.P = 1e5
d.N = 10
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.2,0.8])
d.FT =320.
d.D = 0.25*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10.


In [11]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [12]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_scaled_minmax = np.min(bp_scaled), np.max(bp_scaled)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled_minmax,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [13]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [14]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [15]:
eqs_wrapped = wrap(eqs)

In [16]:
sol=minimize(lambda x: 0., x, method='SLSQP', bounds=[(-20,20)]*len(x), constraints=dict(type='eq',fun=eqs_wrapped))

In [17]:
res=xtod(sol.x,d2)

In [18]:
eqs(xtod(sol.x,d2))

(array([ 3.20142135e-10,  1.27329258e-10,  2.18278728e-10,  6.91215973e-11,
        -1.60071068e-10,  8.36735126e-11, -5.82076609e-11,  2.25554686e-10,
         3.49245965e-10, -1.74622983e-10,  1.01863407e-10,  2.54658516e-10,
        -1.32786226e-10, -6.69388101e-10,  8.64019967e-12, -4.36557457e-11,
        -4.74642547e-12, -1.16415322e-10,  6.53344046e-12,  3.99013516e-08,
         5.68434189e-14, -2.84217094e-14, -8.52651283e-14,  1.42108547e-14,
        -2.84217094e-14,  1.42108547e-14,  1.13686838e-13,  1.42108547e-14,
        -5.68434189e-14,  2.84217094e-14,  1.42108547e-14, -8.52651283e-14,
         1.42108547e-14,  5.68434189e-14,  1.50990331e-14, -3.41060513e-13,
        -1.12132525e-14,  2.84217094e-13, -1.38777878e-17,  5.68434189e-14,
         0.00000000e+00,  5.21540642e-08,  2.23517418e-08, -7.45058060e-08,
         2.60770321e-08,  0.00000000e+00,  7.45058060e-09, -2.23517418e-08,
         1.56462193e-07,  3.48314643e-07, -1.74622983e-10,  0.00000000e+00,
         2.0